In [1]:
import os
import time
import numpy as np
import soundfile as sf
import librosa
import csv
import scipy.misc
import imageio
import shutil
import sys
import datetime
import random
import collections
import pickle
import math
from random import randint

from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
# set the memory usage
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
set_session(tf.Session(config=tf_config))

from keras.layers import Dropout, AveragePooling2D,LSTM,Reshape, Bidirectional
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, Input, BatchNormalization
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras import losses, optimizers
#from keras.layers.pooling import GlobalMaxPooling1D

Using TensorFlow backend.


In [2]:
with open("datasets/train_5000_babycry_Prob1_44100_128_framelevel_calssifier.pkl", "rb") as input_file:
    train_data = pickle.load(input_file)
input_file.close()

In [3]:
train_frames = train_data['train_frames']
train_labels = train_data['train_labels']

In [4]:
len(train_frames)

5000

In [5]:
with open("datasets/val_1000_babycry_44100_128_framelevel_calssifier.pkl", "rb") as input_file:
    val_data = pickle.load(input_file)
input_file.close()

In [6]:
val_frames = val_data['val_frames']
val_labels = val_data['val_labels']

In [7]:
len(val_frames)

496

In [8]:
np.array(train_frames).shape

(5000, 128, 200)

In [9]:
train_frames = np.array(train_frames).reshape(5000,128,200,1)
val_frames = np.array(val_frames[0:500]).reshape(496,128,200,1)
train_labels = np.array(train_labels).reshape(5000,200)
val_labels = np.array(val_labels[0:500]).reshape(496,200)

In [10]:
def get_1d_conv_model():
    
    inp = Input(shape=(128,200,1))
    
    x = Conv2D(64, (10,1), strides=(1,1) , padding="same")(inp)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((10,1),strides=(10,1))(x)
    x = Dropout(rate=0.1)(x)
    print(x._keras_shape)
    
    x = Conv2D(256, (7,1), strides=(1,1) , padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((7,1),strides=(7,1))(x)
    x = Dropout(rate=0.1)(x)
    print(x._keras_shape)
    
    
    x = Reshape((256,200))(x)
    
    x = LSTM(200, return_sequences=True, return_state=True)(x)
    
    x = LSTM(200, return_sequences=True)(x)
    
    
    print(x._keras_shape)
    x = Flatten()(x)
    out = Dense(200, activation='sigmoid')(x)
    
    model = Model(inputs=inp, outputs=out)
    opt = optimizers.Adam(lr=0.001)

    model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=['acc'])
    return model

In [11]:
my_model = get_1d_conv_model()

(None, 12, 200, 64)
(None, 1, 200, 256)
(None, 256, 200)


In [12]:
my_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 200, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 200, 64) 704         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128, 200, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 128, 200, 64) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [13]:
earlystop = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=10, mode='auto')

In [ ]:
my_model.fit(x=train_frames,y=train_labels,batch_size=150,epochs=100, 
             validation_data=(val_frames, val_labels), callbacks=[earlystop])

Train on 5000 samples, validate on 496 samples
Epoch 1/100


In [ ]:
my_model = load_model('final_melspec_babycry_model_2cnn_64_256_2lstm_1dense_200_01.h5')

In [15]:
with open('test_set_2_500_babycry_44100_128_multiple_estimated_11_preprocessed_with_confidence0.6.txt','r') as regions_file:
    reader = (regions_file.read())
    l = list(reader.split('\n'))
regions_file.close()

In [5]:
l[0]

'mixture_devtrain_babycry_001_33936452bb1d5ace9cc204637efc0512.wav;16.689655172413794;18.20689655172414;babycry;0.9950682'

In [19]:
#my_model.save('final_melspec_babycry_model_2cnn_64_256_2lstm_1dense_200_05.h5')

/usr/local/lib/python3.5/dist-packages/keras/engine/topology.py:2368: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 200) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 200) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [6]:
#creating unified-length segments out of the proposed regions by Mask R-CNN

frames_per_sample = 200

def create_melspec_test(path,start_time,finish_time):
    test_frames=[]
    
    melspec = imageio.imread(path)
    print(path)
    event_start_index = int(np.round(float(start_time)/30 * 1305))
    event_finish_index = int(np.round(float(finish_time)/30 * 1305))
    event_length = event_finish_index - event_start_index 
    print(event_start_index)
    print(event_finish_index)
                    
    #calculating sample start and finish indices, randomly
    sample_start = randint(max(0,event_start_index - max(0,(200-event_length))),event_start_index)
    sample_finish = min(1305,sample_start+frames_per_sample)
                    
    print('before')
    print(sample_start)
    print(sample_finish)
    sample_length = sample_finish - sample_start 
    if sample_length < frames_per_sample:#if we don't have enough negative samples after event
        sample_start = sample_start - (frames_per_sample - sample_length)
                    
    print('after')
    print(sample_start)
    print(sample_finish)
                    
    temp1 = np.array(melspec)
    temp_test_frames = temp1[:,sample_start:sample_finish]
                    
    print(np.array(temp_test_frames).shape)
    
    temp2 = np.array(temp_test_frames)
    temp2 = temp2.reshape(128,200)
    test_frames = temp2
                    
    print(np.array(test_frames).shape)
    
    
    return test_frames

In [7]:
def choose_most_probable_event(dic):
    delimeter = '\t'
    ind = 0
    max_val = 0
    max_above1 = 0
    
    for i in range(len(dic['predicted_values'])):
        temp_list = dic['predicted_values'][i][0]
        conf = dic['maskrcnn_conf'][i]
        #print(temp_list)
        print('conf is '+str(conf))
        
        m = max(temp_list)
        sum_above1 = 0
        num_above1 = 0
        for j in temp_list:
            if j>=0.1:
                sum_above1 += j
                num_above1 += 1
        if num_above1 > 0:
            avg_above1 = float(sum_above1/num_above1)
        else:
            avg_above1 =0 
            
        if (0.7*(avg_above1) + 0.3*(conf)) > max_val:
            max_val = 0.7*(avg_above1) + 0.3*(conf)
            max_above1 = avg_above1
            max_m = m
            ind = i
        print('avg above 1 is '+str(avg_above1))
        #print('0.7*(avg_above1) + 0.3*(conf) is '+str(0.7*(avg_above1) + 0.3*(conf)))
             
        
    if max_val >= 0.8:
        st = dic['file_name'][ind]+ delimeter + str(dic['event_start_time'][ind]) + delimeter + str(dic['event_end_time'][ind]) + delimeter + event
    else:
        print(ind)
        st = dic['file_name'][ind]+ delimeter + '0.0\t0.0\tno_event'
    return st
    

In [8]:
logs_path = 'logs/'
log_path = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')
log_path = os.path.join(logs_path, log_path)

if os.path.exists(log_path):
    pass
else:
    os.mkdir(log_path)
shutil.copyfile('LogMel_Classifier-with_context_babycry.ipynb', os.path.join(log_path, 'logmel_classifier_code.ipynb'))
 

'logs/2018-11-30-16-12/logmel_classifier_code.ipynb'

In [25]:
event = 'babycry'
test_path = '../Maskrcnn_audio/datasets/test_set_2_babycry/'
previous_file_name = ''
dic = collections.defaultdict(list)


#with open(os.path.join(log_path+'/final_report.txt'),'w') as w_file:
with open(os.path.join(log_path + '/final_report.txt'),'w') as w_file:
    for element in l:
        parts = element.split(';')
        print(parts)
        if(parts[3]==event):
            
            current_file_name = parts[0].replace('[','').replace("'",'')
            if previous_file_name == '':
                previous_file_name = current_file_name
            start_time = float(parts[1])
            finish_time = float(parts[2])
            
            
            test_frames = create_melspec_test(test_path + current_file_name +'.png', start_time, finish_time)
            test_frames = np.array(test_frames)
            test_frames = test_frames.reshape(1,128,200,1)
            print(test_frames.shape)
            
            predictions = my_model.predict(x=test_frames)
            
            print(parts[0]+ parts[1]+parts[2])
            print(predictions)
                    
            predict_values = np.array(predictions)
            if current_file_name == previous_file_name:
                dic['file_name'].append(current_file_name)
                dic['event_start_time'].append(float(parts[1]))
                dic['event_end_time'].append(float(parts[2]))
                dic['predicted_values'].append(predict_values)
                dic['maskrcnn_conf'].append(float(parts[4]))
                print('***')
            else:
                
                st = choose_most_probable_event(dic)
                w_file.write(st+'\n')
                dic.clear()
                previous_file_name = current_file_name
                dic['file_name'].append(current_file_name)
                dic['event_start_time'].append(float(parts[1]))
                dic['event_end_time'].append(float(parts[2]))
                dic['predicted_values'].append(predict_values)
                dic['maskrcnn_conf'].append(float(parts[4]))
                
        else:
            w_file.write(element+'\n')
    #for the last file
    st = choose_most_probable_event(dic)
    w_file.write(st+'\n')
    dic.clear()    
            
w_file.close()

['mixture_devtrain_babycry_001_33936452bb1d5ace9cc204637efc0512.wav', '16.689655172413794', '18.20689655172414', 'babycry', '0.9950682']
../Maskrcnn_audio/datasets/test_set_2_babycry/mixture_devtrain_babycry_001_33936452bb1d5ace9cc204637efc0512.wav.png
726
792
before
622
822
after
622
822
(128, 200)
(128, 200)
(1, 128, 200, 1)
mixture_devtrain_babycry_001_33936452bb1d5ace9cc204637efc0512.wav16.68965517241379418.20689655172414
[[7.00074628e-14 1.02302776e-14 1.13361155e-14 1.74924813e-13
  1.96844168e-15 2.32935221e-12 5.63099318e-13 4.39403982e-12
  1.95655384e-11 2.34037876e-12 2.26611546e-12 3.00532971e-12
  8.25470040e-11 6.19632054e-11 9.13482978e-09 6.38417532e-08
  2.95556664e-08 5.93253944e-08 5.38099023e-08 1.70555473e-10
  1.10500498e-11 3.30838724e-11 2.88475093e-07 1.14761718e-07
  5.16806040e-06 7.41291979e-06 4.52233917e-06 3.67877262e-09
  4.14206681e-07 1.25345889e-06 4.33359482e-09 2.95917513e-09
  6.85882904e-12 1.78247625e-10 1.13216839e-10 1.52649782e-10
  2.70902242

mixture_devtrain_babycry_006_7ff28116e1688b9185580ef61cf7a709.wav24.29885057471264525.17241379310345
[[5.93326099e-09 1.32665920e-10 3.42564768e-08 3.86807919e-08
  3.50084939e-08 7.43740847e-09 2.52245314e-07 1.16359422e-06
  1.54384161e-05 2.75793682e-05 8.97306123e-08 4.83045132e-11
  7.63869037e-11 6.00989397e-12 1.50758106e-09 1.64239943e-12
  1.02193080e-12 1.88705686e-13 3.27037659e-13 3.62863524e-11
  5.32885693e-13 1.24173360e-12 4.01180738e-12 4.53809178e-11
  6.10862446e-12 3.87520016e-11 2.79091637e-07 5.44868144e-11
  1.38038123e-10 1.16531112e-12 1.56059714e-11 8.63516547e-09
  6.98626792e-08 1.12679177e-09 3.17215726e-10 1.84769160e-08
  1.00848698e-08 1.51825592e-08 5.90792865e-07 1.06191067e-06
  2.87679445e-08 2.74567753e-07 6.02077648e-07 1.03083421e-05
  5.29674253e-07 5.20832444e-09 6.16403071e-08 1.67337444e-09
  2.02058570e-09 8.87161455e-09 9.62469771e-10 1.76318890e-10
  6.53672186e-11 3.73970438e-10 1.95457428e-10 2.85622122e-07
  7.15160127e-08 2.31817808e-06

mixture_devtrain_babycry_021_43d2e5fdcd3ed845f1582c8bfccd3525.wav16.98850574712643519.03448275862069
[[1.00888454e-13 4.51253618e-14 3.30030409e-13 1.69095654e-13
  5.56942567e-13 5.28537819e-14 1.63914712e-13 6.16771117e-16
  1.45575190e-15 1.67070797e-17 5.06293678e-18 4.23548760e-19
  3.92890773e-17 2.89066934e-17 8.16618429e-16 1.99663352e-13
  1.72763615e-09 1.14841860e-07 1.73132157e-08 1.45993093e-08
  5.34026101e-10 1.51999444e-08 1.89487911e-10 7.35980027e-11
  2.57496802e-11 1.03319533e-10 6.21605636e-07 1.95343954e-08
  7.65299092e-07 2.71604534e-07 1.22696849e-06 1.57932122e-11
  8.29398147e-11 5.81586049e-13 2.14328598e-12 3.44739723e-11
  9.14548758e-15 1.06439753e-14 1.64434877e-16 1.32057231e-14
  9.27126865e-13 3.13136392e-14 7.53295284e-15 4.78330923e-14
  1.21950010e-14 2.13694046e-14 2.69587195e-12 7.06111766e-11
  3.58379132e-10 3.03577337e-11 7.82816312e-11 2.83387147e-10
  5.44821410e-11 1.95938557e-10 2.59051758e-09 2.20429630e-09
  4.08043807e-11 3.06811798e-09

mixture_devtrain_babycry_024_e0990701c23c1d1e441ce299ee86cb2b.wav26.11494252873563228.988505747126435
[[1.1432793e-13 9.1517397e-14 9.6293893e-12 5.5099081e-09 7.9612803e-13
  1.1497261e-09 1.7722264e-12 1.9704347e-13 1.8912664e-12 3.0031089e-14
  2.4544237e-15 2.4192651e-17 3.6330927e-16 1.0034539e-16 1.3904608e-12
  3.3495006e-14 2.2880507e-10 5.7665076e-13 7.0878546e-13 5.8878732e-13
  4.3156703e-13 1.0997866e-14 1.2083764e-15 2.1354663e-17 4.6893297e-12
  3.4989952e-11 1.4030180e-10 2.8064383e-14 1.0712828e-12 1.0120315e-10
  1.3974814e-10 3.7093544e-11 1.0358390e-11 4.3509338e-11 7.5884415e-08
  4.9013682e-10 4.5907271e-11 1.1327504e-12 8.8016030e-16 2.6955982e-13
  2.3784395e-13 2.6279237e-15 2.7513108e-11 4.6491155e-10 6.6338157e-10
  9.3713010e-11 8.6589523e-08 6.7519329e-10 3.1002034e-07 1.4946282e-07
  8.4954809e-07 5.8900501e-10 2.4898202e-07 5.0861718e-08 2.2551651e-06
  8.0496343e-03 2.4741665e-03 9.0009365e-03 2.0242967e-09 6.2549454e-01
  9.9990737e-01 1.0000000e+00 1.00

mixture_devtrain_babycry_028_6e20438df474ea7f74d9670283c637cc.wav20.8965517241379323.195402298850574
[[6.79213219e-09 6.81059298e-09 1.46459900e-09 5.28789235e-09
  1.11020825e-07 1.52767630e-08 3.44754336e-10 2.26778249e-10
  2.75565295e-08 1.26196417e-12 3.48623311e-14 5.05846551e-15
  3.30754304e-14 1.22679373e-13 1.95240057e-15 7.66053818e-17
  7.17185030e-18 9.53552084e-15 1.15541020e-14 6.15790513e-13
  2.20976200e-15 5.79898407e-15 5.46043442e-16 5.23574102e-18
  1.61066306e-16 9.29181167e-15 2.15787827e-14 2.54092816e-15
  2.61663277e-13 2.87129001e-12 7.08966330e-10 3.28511690e-10
  4.98371934e-11 8.89499932e-12 3.97741475e-12 6.58261119e-12
  2.15552853e-11 5.74991860e-11 1.56197144e-09 2.16404583e-09
  1.58947988e-09 3.73796237e-11 1.77132755e-12 9.55291144e-14
  1.64470604e-09 8.60768401e-10 1.14290233e-09 1.83977833e-09
  1.05524910e-13 7.09625804e-12 2.18887325e-10 8.58782911e-09
  9.40660216e-10 5.21969403e-12 3.53133842e-11 1.96690531e-09
  1.08198783e-08 3.63933668e-06

...

mixture_devtrain_babycry_485_826e9f3a8a818e0fab236b8ca04ce6cb.wav9.83908045977011613.241379310344827
[[9.28205490e-01 9.93896663e-01 9.98247266e-01 9.99228120e-01
  9.98698831e-01 9.99856591e-01 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00

mixture_devtrain_babycry_488_0d14828d8b9a633780fb0bd6aacc49ec.wav15.1724137931034517.448275862068964
[[3.88362204e-10 2.22718176e-12 2.07046846e-09 6.52848609e-11
  1.54749876e-11 3.87583325e-12 9.05245545e-10 4.02189571e-09
  1.47784673e-09 1.84719295e-10 3.92462229e-10 9.81553380e-11
  3.73631421e-12 1.46527289e-12 6.79450593e-12 9.35426459e-11
  2.56694079e-06 1.49241497e-09 5.02704856e-09 4.74397500e-11
  2.73871095e-13 9.49173726e-11 1.14742528e-10 6.54142365e-11
  2.79017209e-09 1.41922876e-10 2.79893464e-09 1.79992234e-08
  2.97497706e-08 2.13361005e-07 2.92155192e-07 4.33631531e-09
  1.23092592e-09 9.15107295e-11 6.99729341e-09 1.04228244e-10
  5.31058086e-10 2.76611422e-06 3.84524327e-07 8.01625475e-03
  4.58783779e-06 6.64433464e-02 4.85545173e-02 9.87510443e-01
  4.94886428e-01 9.99919772e-01 9.99975324e-01 9.99960423e-01
  9.99999881e-01 9.99999523e-01 1.00000000e+00 9.99999762e-01
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00

mixture_devtrain_babycry_492_8ee8cc815023dad3006b0646f5fd0ad6.wav9.31034482758620612.528735632183908
[[3.3071411e-13 1.4789942e-13 1.5734836e-13 9.7155050e-16 1.3705195e-14
  2.9453125e-16 1.1601549e-08 1.7835653e-09 3.5254834e-11 5.7138482e-08
  2.5310176e-06 7.0402812e-06 8.8215688e-07 7.8902331e-06 2.8061685e-08
  5.4794042e-03 1.3625381e-03 2.6507931e-02 9.8595154e-01 5.8134758e-01
  2.0688392e-02 9.9998772e-01 9.9728930e-01 9.9999893e-01 9.9997854e-01
  9.9998355e-01 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.000

mixture_devtrain_babycry_502_7d5981ab80af98d8da3d0108cca864a3.wav9.83908045977011613.28735632183908
[[4.37151509e-10 2.27710647e-17 8.99137449e-12 2.47212548e-16
  9.62192288e-18 1.02989756e-18 9.07701852e-15 1.07193415e-13
  1.94459466e-13 3.09126982e-15 7.54728040e-14 9.96725827e-14
  2.62319666e-15 5.84406253e-16 3.61047319e-14 1.36660349e-15
  1.00566013e-18 1.78633468e-15 6.94159834e-12 9.13456702e-13
  4.94862440e-15 4.48134724e-15 1.18210065e-16 8.09178515e-18
  5.87564733e-16 3.74897735e-16 2.99920223e-12 1.29472713e-13
  2.72855449e-09 3.29723714e-11 4.94034466e-11 1.31000020e-11
  2.25659866e-10 3.66506088e-12 1.80004310e-11 4.13418882e-10
  5.48819887e-12 9.02464681e-14 7.70306596e-11 9.56976805e-07
  1.61521960e-08 1.07086855e-07 8.55965726e-03 9.98458266e-01
  9.99999166e-01 9.99873400e-01 9.97567654e-01 9.99999642e-01
  1.00000000e+00 9.99995351e-01 9.99999523e-01 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 

mixture_devtrain_babycry_509_23f1ebc8c8f69b66382706dfecc6a2df.wav19.14942528735632320.436781609195403
[[2.87507990e-10 3.80424418e-14 3.50492641e-15 4.94720604e-16
  1.09208170e-14 9.86899157e-16 1.49280138e-14 3.10833478e-14
  4.74441580e-12 9.66229185e-14 1.31833680e-15 6.83326095e-16
  4.33499807e-14 2.44817721e-15 3.28257083e-12 4.25369002e-14
  3.48212279e-12 5.59893416e-11 2.38123937e-10 1.45838051e-12
  7.63072805e-14 5.00149450e-16 3.64265460e-16 2.48061560e-18
  1.69769924e-15 1.29960223e-15 1.21017490e-16 5.24651259e-20
  4.97575867e-19 4.43494488e-19 8.75248568e-16 5.92098199e-17
  7.60583979e-13 5.39856381e-14 8.05991236e-15 5.65782447e-12
  3.16979706e-11 1.26882727e-10 6.59174867e-11 3.01085185e-10
  3.66897094e-12 5.57394452e-12 4.92400676e-10 4.48332074e-08
  1.51591812e-04 1.23592386e-07 2.94133806e-05 7.29757677e-09
  4.31278444e-08 2.89231525e-06 2.41667550e-07 1.73038956e-07
  9.92695732e-07 9.28165662e-07 6.04801357e-07 1.82351325e-08
  2.09271889e-08 1.71068677e-0

In [9]:
import math

In [10]:
def check_event_truth(target_name,start_time):
    with open('event_list_evaltest_gunshot.csv', mode='r') as csv_file:
        reader = list(csv.reader(csv_file, delimiter='\n'))
        
        for row,i in zip(reader,range(500)):
            fields= str(row).split('\\t')
            if len(fields)>1:
                
                if fields[0].replace('[','').replace("'",'') == target_name and fields[3].replace(']','').replace("'",'')==event and abs(float(fields[1])-float(start_time))<=0.5:
                    return True
    
    return False

In [11]:
#gets a file name and return the number in it!
def get_number_from_filename(s):
    p = s.split('_')
    return p[3]

In [22]:
#creating file with probability feature values
event = 'gunshot'
test_path = '../Maskrcnn_audio/datasets/eval_500_gunshot_44100_128/'
with open('eval_regions_melspec_44100_128_37_models_features_3cnn_1bidirectionLSTM_64.csv','w') as w_file:
    w_writer = csv.writer(w_file, delimiter='\t', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    w_writer.writerow(['file_name', 'start', 'finish','min_val', 'max_val','average','average of probs>=0.9',
                       'average of probs>=0.8','average of probs>=0.7','average of probs>=0.5','average of probs>=0.1'
                       ,'average over all','True/False','Mask-rcnn confidence','prob>0.9+conf','0.7*prob9+0.3conf',
                       '0.3*prob9+0.7conf','0.7*prob1+0.3*conf',
                               '0.3*prob1+0.7*conf'])
    
    for element in l:
        parts = element.split(';')
        
        if(parts[3]==event):
            current_file_name = parts[0].replace('[','').replace("'",'')
            start_time = float(parts[1])
            finish_time = float(parts[2])
            
            
            test_frames = create_melspec_test(test_path + current_file_name +'.png', start_time, finish_time)
            
            
            test_frames = test_frames.reshape(1,128,200,1)
            print(test_frames.shape)
            
            
            predict_values = []
            predictions = my_model.predict(x=test_frames)
            
            
            print(parts[0]+ parts[1]+parts[2])
            
            predict_values = predictions.tolist()
            
            file_number = get_number_from_filename(current_file_name)
            min_val = str(min(predict_values[0]))
            max_val = str(max(predict_values[0]))
            avg_val = str(sum(predict_values[0])/12)
            sum_above_9 = 0
            num_of_probs_above_9 = 0
            sum_above_8 = 0
            num_of_probs_above_8 = 0
            sum_above_7 = 0
            num_of_probs_above_7 = 0
            sum_above_5 = 0
            num_of_probs_above_5 = 0
            sum_above_1 = 0
            num_of_probs_above_1 = 0
            sum_over_all = 0
            avg_over_all = 0
            
            for i in range(len(predict_values[0])):
                sum_over_all += predict_values[0][i]
                if predict_values[0][i]>= 0.9:
                    sum_above_9 += predict_values[0][i]
                    num_of_probs_above_9 += 1
                if predict_values[0][i]>= 0.8:
                    sum_above_8 += predict_values[0][i]
                    num_of_probs_above_8 += 1
                if predict_values[0][i]>= 0.7:
                    sum_above_7 += predict_values[0][i]
                    num_of_probs_above_7 += 1
                if predict_values[0][i]>= 0.5:
                    sum_above_5 += predict_values[0][i]
                    num_of_probs_above_5 += 1
                if predict_values[0][i]>= 0.1:
                    sum_above_1 += predict_values[0][i]
                    num_of_probs_above_1 += 1
                   
            if num_of_probs_above_9 > 0:
                avg_above_9 = float(sum_above_9 / num_of_probs_above_9)
            else:
                avg_above_9 = 0
            if num_of_probs_above_8 > 0:
                avg_above_8 = float(sum_above_8 / num_of_probs_above_8)
            else:
                avg_above_8 = 0
            if num_of_probs_above_7 > 0:
                avg_above_7 = float(sum_above_7 / num_of_probs_above_7)
            else:
                avg_above_7 = 0
            if num_of_probs_above_5 > 0:
                avg_above_5 = float(sum_above_5 / num_of_probs_above_5)
            else:
                avg_above_5 = 0
            if num_of_probs_above_1 > 0:
                avg_above_1 = float(sum_above_1 / num_of_probs_above_1)
            else:
                avg_above_1 = 0
            avg_over_all = float(sum_over_all/len(predict_values[0]))
            
            region_truth = check_event_truth(current_file_name,parts[1])
            w_writer.writerow([file_number,parts[1],parts[2] , min_val , max_val , avg_val,avg_above_9, 
                               avg_above_8, avg_above_7, avg_above_5,avg_above_1,avg_over_all,region_truth,parts[4],
                               avg_above_9+float(parts[4]),0.7*avg_above_9+(0.3*float(parts[4])),
                               0.3*avg_above_9+(0.7*float(parts[4])),0.7*avg_above_1+(0.3*float(parts[4])),
                               0.3*avg_above_1+(0.7*float(parts[4]))])
            
            
            
        else:
            pass
            
            
w_file.close()